In [2]:
import os
import openai
import sys
sys.path.append('../..')

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.vectorstores import Chroma
from langchain_openai import OpenAIEmbeddings
persist_directory = 'docs/chroma/'

In [4]:
embedding = OpenAIEmbeddings()
vectordb = Chroma(
    persist_directory=persist_directory,
    embedding_function=embedding
)

In [5]:
print(vectordb._collection.count())

38


In [6]:
texts = [
    """The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).""",
    """A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.""",
    """A. phalloides, a.k.a Death Cap, is one of the most poisonous of all known mushrooms.""",
]

In [7]:
smalldb = Chroma.from_texts(texts, embedding=embedding)

In [8]:
question = "Tell me about all-white mushrooms with large fruiting bodies"

In [9]:
smalldb.similarity_search(question, k=2)

[Document(page_content='A mushroom with a large fruiting body is the Amanita phalloides. Some varieties are all-white.'),
 Document(page_content='The Amanita phalloides has a large and imposing epigeous (aboveground) fruiting body (basidiocarp).')]

### Addressing Diversity: Maximum marginal relevance
Last class we introduced one problem: how to enforce diversity in the search results.

Maximum marginal relevance strives to achieve both relevance to the query and diversity among the results.

In [10]:
question = "retaliatory strikes came in response to a drone strike?"
docs_ss = vectordb.similarity_search(question,k=3)

In [11]:
docs_ss[0].page_content[:100]

'The\nUS\nconducted\nmajor\nairstrikes\non\n85\ntargets\nin\nIraq\nand\nSyria\non\nFriday,\nthe\nstart\nof\nwhat\nwill\n'

In [12]:
docs_ss[1].page_content[:100]

'The\nUS\nconducted\nmajor\nairstrikes\non\n85\ntargets\nin\nIraq\nand\nSyria\non\nFriday,\nthe\nstart\nof\nwhat\nwill\n'

Note the difference in results with MMR.

In [13]:
docs_mmr = vectordb.max_marginal_relevance_search(question,k=3)

In [14]:
docs_mmr[0].page_content[:100]

'The\nUS\nconducted\nmajor\nairstrikes\non\n85\ntargets\nin\nIraq\nand\nSyria\non\nFriday,\nthe\nstart\nof\nwhat\nwill\n'

In [15]:
docs_mmr[1].page_content[:100]

'Image'

### Addressing Specificity: working with metadata
In last lecture, we showed that a question about the third lecture can include results from other lectures as well.

To address this, many vectorstores support operations on metadata.

metadata provides context for each embedded chunk.

In [16]:
question = "what facilities were included in the strike?"

In [17]:
docs = vectordb.similarity_search(
    question,
    k=3,
    filter={"source":"docs/cnn article.pdf"}
)

In [18]:
for d in docs:
    print(d.metadata)

{'page': 0, 'source': 'docs/cnn article.pdf'}
{'page': 0, 'source': 'docs/cnn article.pdf'}
{'page': 0, 'source': 'docs/cnn article.pdf'}


### Addressing Specificity: working with metadata using self-query retriever
But we have an interesting challenge: we often want to infer the metadata from the query itself.

To address this, we can use SelfQueryRetriever, which uses an LLM to extract:

1. The query string to use for vector search
2. A metadata filter to pass in as well

Most vector databases support metadata filters, so this doesn't require any new databases or indexes.

In [19]:
from langchain.llms import OpenAI
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain.chains.query_constructor.base import AttributeInfo

In [30]:
metadata_field_info = [
    AttributeInfo(
        name="source",
        description="The article the chunk is from, should be one of `docs/cnn article.pdf`, `docs/nytimes article1.pdf`, or `docs/nytimes article2.pdf`",
        type="string",
    ),
    AttributeInfo(
        name="page",
        description="The page from the article",
        type="integer",
    ),
]

In [35]:
document_content_description = "news article"
llm = OpenAI(model='gpt-3.5-turbo-instruct', temperature=0)
retriever = SelfQueryRetriever.from_llm(
    llm,
    vectordb,
    document_content_description,
    metadata_field_info,
    verbose=True
)

In [36]:
question = "what facilities were included in the strike??"

In [37]:
docs = retriever.get_relevant_documents(question)

In [38]:
for d in docs:
    print(d.metadata)

{'page': 0, 'source': 'docs/cnn article.pdf'}
{'page': 0, 'source': 'docs/cnn article.pdf'}
{'page': 1, 'source': 'docs/cnn article.pdf'}
{'page': 1, 'source': 'docs/cnn article.pdf'}
